In [3]:
#!/usr/bin/env python3
"""
Stegano Training + Comprehensive Evaluation (reduced metrics)
- Keeps: PSNR, SSIM, NRPC, USEI (UQI), LPIPS (optional)
- Adds: Recovery bitstream accuracy, Residual-signal estimated capacity (bpp)
- Adds separate graphs for every metric.
- Usage: same as your original script — ensure dataset path correct.
"""

import os, math, torch, torch.nn as nn, torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms, models
from torchvision.utils import make_grid, save_image
from pytorch_msssim import ssim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
from PIL import Image
from glob import glob
from datetime import datetime
import pandas as pd
from skimage.metrics import structural_similarity as ssim_sk

# Optional / best-effort imports
has_lpips = False
try:
    import lpips
    has_lpips = True
except Exception:
    print("lpips not available — LPIPS will be skipped. Install with 'pip install lpips'")

# -----------------------------
# Config
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
torch.manual_seed(42)

img_size = 128
batch_size = 24
lr = 1e-4
epochs = 50
lambda_ssim = 1.0
lambda_rec = 15.0
lambda_perc = 0.5
save_every = 5
DATA_FOLDER = "img_128x_50000"

print("Device:", device, "img_size:", img_size)

# -----------------------------
# Dataset
# -----------------------------
class CustomImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        patterns = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]
        self.paths = []
        for p in patterns:
            self.paths += sorted(glob(os.path.join(folder_path, "*" + p)))
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform: img = self.transform(img)
        return img, 0

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(0.1,0.1,0.1,0.02),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

dataset = CustomImageDataset(DATA_FOLDER, transform=transform)
print("Loaded images:", len(dataset))

# -----------------------------
# Utility metrics
# -----------------------------
def psnr_t(a, b, data_range=2.0):
    mse_val = F.mse_loss(a, b).item()
    if mse_val == 0: return float("inf")
    return 10 * math.log10((data_range ** 2) / mse_val)

def tensor_to_vgg_input(x):
    x = (x + 1) / 2
    mean = torch.tensor([0.485, 0.456, 0.406]).to(x.device).view(1,3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).to(x.device).view(1,3,1,1)
    return (x - mean) / std

def compute_nrpc(cover, stego):
    c = ((cover + 1) / 2).detach().cpu().numpy()
    s = ((stego + 1) / 2).detach().cpu().numpy()
    res_c = c - np.mean(c)
    res_s = s - np.mean(s)
    num = np.sum(res_c * res_s)
    denom = np.sqrt(np.sum(res_c ** 2) * np.sum(res_s ** 2)) + 1e-12
    return float(num / denom)

def compute_usei_uqi(a, b):
    a_np = ((a + 1) / 2).detach().cpu().numpy()
    b_np = ((b + 1) / 2).detach().cpu().numpy()
    Bs = a_np.shape[0]
    vals = []
    for i in range(Bs):
        uqi_ch = []
        for ch in range(a_np.shape[1]):
            x = a_np[i,ch].flatten()
            y = b_np[i,ch].flatten()
            mean_x = x.mean(); mean_y = y.mean()
            cov = np.mean((x-mean_x)*(y-mean_y))
            var_x = x.var(); var_y = y.var()
            num = 4 * mean_x * mean_y * cov
            den = (mean_x**2 + mean_y**2) * (var_x + var_y) + 1e-12
            uqi = num/den
            uqi_ch.append(uqi)
        vals.append(np.mean(uqi_ch))
    return float(np.mean(vals))

def image_to_uint8(t):
    # t expected in [-1,1]; convert -> [0,255] uint8
    with torch.no_grad():
        x = ((t + 1) / 2.0 * 255.0).round().clamp(0,255).to(torch.uint8).cpu().numpy()
    return x  # shape: (B, C, H, W), dtype=uint8

def batch_bitstream_accuracy(secret_t, recovered_t):
    """
    Convert secret and recovered images to 8-bit/channel bitstreams and compute bit accuracy.
    Returns (matches / total_bits) as float.
    """
    s_uint8 = image_to_uint8(secret_t)
    r_uint8 = image_to_uint8(recovered_t)
    # pack bits per byte (big-endian) -> shape (..., 8)
    # flatten across batch, channels, height, width
    s_flat = s_uint8.reshape(-1).astype(np.uint8)
    r_flat = r_uint8.reshape(-1).astype(np.uint8)
    # convert to bits
    s_bits = np.unpackbits(s_flat)
    r_bits = np.unpackbits(r_flat)
    # safety: lengths same
    total_bits = s_bits.size
    if total_bits == 0:
        return float('nan')
    matches = int(np.sum(s_bits == r_bits))
    return matches / total_bits

def residual_bpp_estimate(cover, stego):
    """
    Estimate residual bit-capacity (bpp) from residual distribution.
    - residual = stego - cover  (both in [-1,1])
    - map residual to 0..255 and compute Shannon entropy (bits per channel symbol)
    - multiply by number of channels (3) => bits per pixel (bpp)
    """
    with torch.no_grad():
        resid = (stego - cover).detach().cpu().numpy()  # shape (B,C,H,W)
    # map resid from [-2,2] into [0,255] (since st ranges [-1,1], resid in [-2,2])
    mapped = ((resid + 2.0) / 4.0 * 255.0).round().clip(0,255).astype(np.uint8).ravel()
    if mapped.size == 0:
        return float('nan')
    hist = np.bincount(mapped, minlength=256).astype(np.float64)
    probs = hist / (hist.sum() + 1e-12)
    probs = probs[probs > 0]
    entropy_bits = -np.sum(probs * np.log2(probs))  # bits per symbol (channel)
    # bits per pixel = entropy_bits * channels (3)
    bpp = float(entropy_bits * 3.0)
    return bpp

def compute_lpips(a, b, lpips_model=None):
    if not has_lpips or lpips_model is None: return float('nan')
    with torch.no_grad(): return float(lpips_model(a, b).mean().item())

# -----------------------------
# Model definitions (same)
# -----------------------------
class CSPBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        split_point = max(in_channels // 2, 1)
        self.split_point = split_point
        self.part1 = nn.Sequential(
            nn.Conv2d(split_point, split_point, 3, padding=1, bias=False),
            nn.BatchNorm2d(split_point),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(split_point, split_point, 3, padding=1, bias=False),
            nn.BatchNorm2d(split_point),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.transition = nn.Sequential(
            nn.Conv2d(split_point + (in_channels - split_point), out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )
    def forward(self, x):
        sp = self.split_point
        x1, x2 = x[:, :sp], x[:, sp:]
        y1 = self.part1(x1)
        return self.transition(torch.cat((y1, x2), dim=1))

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(6, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.csp1 = CSPBlock(64, 128)
        self.down1 = nn.Sequential(
            nn.Conv2d(128, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.csp2 = CSPBlock(128, 128)
        self.csp3 = CSPBlock(128, 64)
        self.refine = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.final = nn.Sequential(nn.Conv2d(64, 3, 1), nn.Tanh())
    def forward(self, cover, secret):
        x = torch.cat((cover, secret), dim=1)
        x = self.initial(x)
        x = self.csp1(x)
        x = self.down1(x)
        x = self.csp2(x)
        x = F.interpolate(x, scale_factor=2, mode="bilinear", align_corners=False)
        x = self.csp3(x)
        x = self.refine(x)
        return self.final(x)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.AdaptiveAvgPool2d((4,4)),
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1)
        )
    def forward(self, x): return self.main(x)

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.conv(x)

class UNetDecoder(nn.Module):
    def __init__(self, in_channels=3, base=64):
        super().__init__()
        self.enc1 = ConvBlock(in_channels, base)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(base, base*2)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(base*2, base*4)
        self.pool3 = nn.MaxPool2d(2)
        self.bottleneck = ConvBlock(base*4, base*8)
        self.up1 = nn.ConvTranspose2d(base*8, base*4, 2, 2)
        self.dec1 = ConvBlock(base*8, base*4)
        self.up2 = nn.ConvTranspose2d(base*4, base*2, 2, 2)
        self.dec2 = ConvBlock(base*4, base*2)
        self.up3 = nn.ConvTranspose2d(base*2, base, 2, 2)
        self.dec3 = ConvBlock(base*2, base)
        self.final_conv = nn.Conv2d(base, 3, 1)
    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        e3 = self.enc3(self.pool2(e2))
        b = self.bottleneck(self.pool3(e3))
        d1 = self.up1(b)
        if d1.size(2)!=e3.size(2): e3 = F.interpolate(e3,size=d1.shape[2:])
        d1 = self.dec1(torch.cat([d1,e3],dim=1))
        d2 = self.up2(d1)
        if d2.size(2)!=e2.size(2): e2 = F.interpolate(e2,size=d2.shape[2:])
        d2 = self.dec2(torch.cat([d2,e2],dim=1))
        d3 = self.up3(d2)
        if d3.size(2)!=e1.size(2): e1 = F.interpolate(e1,size=d3.shape[2:])
        d3 = self.dec3(torch.cat([d3,e1],dim=1))
        return torch.tanh(self.final_conv(d3))

class VGGPerceptual(nn.Module):
    def __init__(self, device):
        super().__init__()
        vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).features.to(device).eval()
        for p in vgg.parameters(): p.requires_grad = False
        self.vgg = vgg; self.layers = [4,9,16,23]
    def forward(self, x):
        feats=[]
        for i,l in enumerate(self.vgg):
            x=l(x)
            if i in self.layers: feats.append(x)
        return feats

# -----------------------------
# Initialize & train loop
# -----------------------------
G = Generator().to(device)
D = Discriminator().to(device)
Dec = UNetDecoder(in_channels=3, base=64).to(device)
VGG = VGGPerceptual(device)

print("Models initialized")

# Data split & loaders
val_size = int(0.2 * len(dataset))
train_ds, val_ds = random_split(dataset, [len(dataset)-val_size, val_size])
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

opt_G = optim.Adam(G.parameters(), lr=lr, betas=(0.5,0.999))
opt_D = optim.Adam(D.parameters(), lr=lr, betas=(0.5,0.999))
opt_Dec = optim.Adam(Dec.parameters(), lr=lr, betas=(0.5,0.999))

sched_G = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_G, 'max', factor=0.5, patience=4)
sched_Dec = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_Dec, 'max', factor=0.5, patience=4)

mse_loss = nn.MSELoss()       # for LSGAN
l1 = nn.L1Loss()
mse = nn.MSELoss()

scaler_G = GradScaler()
scaler_D = GradScaler()
scaler_Dec = GradScaler()

# LPIPS model init (if available)
lpips_model = None
if has_lpips:
    try:
        lpips_model = lpips.LPIPS(net='vgg').to(device)
    except Exception:
        lpips_model = None

# -----------------------------
# Training loop (with new eval metrics)
# -----------------------------
best_val_psnr = -1.0
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f"runs_steg_{timestamp}"
os.makedirs(log_dir, exist_ok=True)

# Logging containers
train_D_losses, train_G_losses, train_Dec_losses = [], [], []
val_psnr_hist, val_ssim_hist = [], []
per_epoch_metrics = []

print("Starting training...")
for epoch in range(1, epochs+1):
    G.train(); D.train(); Dec.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}")
    iter_idx = 0
    for cover, _ in pbar:
        cover = cover.to(device)
        secret = torch.flip(cover, dims=[-1]).to(device)

        bs = cover.size(0)
        real_label = torch.full((bs,1), 0.9, device=device)
        fake_label = torch.zeros((bs,1), device=device)

        # ---- Train D ----
        opt_D.zero_grad()
        with autocast():
            with torch.no_grad():
                gen_out = G(cover, secret)
                stego_fake = torch.clamp(cover + 0.1 * gen_out, -1, 1)
            pred_real = D(cover)
            pred_fake = D(stego_fake)
            D_loss = 0.5 * (mse_loss(pred_real, real_label) + mse_loss(pred_fake, fake_label))
        scaler_D.scale(D_loss).backward(); scaler_D.step(opt_D); scaler_D.update()

        # ---- Train G ----
        opt_G.zero_grad()
        for p in Dec.parameters(): p.requires_grad = False
        with autocast():
            gen_out = G(cover, secret)
            stego = torch.clamp(cover + 0.1 * gen_out, -1, 1)
            pred = D(stego)
            adv_loss = mse_loss(pred, real_label)
            ssim_loss = 1 - ssim(stego, cover, data_range=2.0, size_average=True)
            recovered = Dec(stego)
            rec_loss = l1(recovered, secret)
            v_stego = VGG(tensor_to_vgg_input(stego))
            v_cover = VGG(tensor_to_vgg_input(cover))
            perc1 = sum(mse(a,b) for a,b in zip(v_stego, v_cover))
            v_rec = VGG(tensor_to_vgg_input(recovered))
            v_secret = VGG(tensor_to_vgg_input(secret))
            perc2 = sum(mse(a,b) for a,b in zip(v_rec, v_secret))
            perc_loss = (perc1 + perc2)
            G_loss = 0.6 * adv_loss + lambda_ssim * ssim_loss + lambda_rec * rec_loss + lambda_perc * perc_loss
        scaler_G.scale(G_loss).backward(); scaler_G.step(opt_G); scaler_G.update()
        for p in Dec.parameters(): p.requires_grad = True

        # ---- Train Decoder ----
        opt_Dec.zero_grad()
        with autocast():
            rec_pred = Dec(stego.detach())
            Dec_loss = l1(rec_pred, secret) + 0.1 * sum(mse(a,b) for a,b in zip(VGG(tensor_to_vgg_input(rec_pred)), VGG(tensor_to_vgg_input(secret))))
        scaler_Dec.scale(Dec_loss).backward(); scaler_Dec.step(opt_Dec); scaler_Dec.update()

        pbar.set_postfix({"D_loss": f"{D_loss.item():.4f}", "G_loss": f"{G_loss.item():.4f}", "Dec_loss": f"{Dec_loss.item():.4f}"})

        train_D_losses.append(D_loss.item())
        train_G_losses.append(G_loss.item())
        train_Dec_losses.append(Dec_loss.item())

        iter_idx += 1

    # ---- Validation ----
    G.eval(); Dec.eval()
    val_psnrs = []
    val_ssims = []
    val_nrpcs = []
    val_useis = []
    val_lpips = []
    val_recovery_accs = []
    val_bpps = []
    with torch.no_grad():
        for j, (vcov, _) in enumerate(val_loader):
            if j >= 50: break
            vcov = vcov.to(device)
            vsecret = torch.flip(vcov, dims=[-1]).to(device)
            gen_out = G(vcov, vsecret)
            vstego = torch.clamp(vcov + 0.1 * gen_out, -1, 1)
            vrecovered = Dec(vstego)

            # PSNR / SSIM / NRPC / USEI
            val_psnrs.append(psnr_t(vrecovered, vsecret, 2.0))
            try:
                val_ssims.append(ssim(vrecovered, vsecret, data_range=2.0, size_average=True).item())
            except Exception:
                # fallback to skimage SSIM per image
                a = ((vrecovered+1)/2).cpu().numpy()
                b = ((vsecret+1)/2).cpu().numpy()
                try:
                    ssum = np.mean([ssim_sk(a[i].transpose(1,2,0), b[i].transpose(1,2,0), multichannel=True) for i in range(a.shape[0])])
                    val_ssims.append(ssum)
                except Exception:
                    pass
            val_nrpcs.append(compute_nrpc(vcov, vstego))
            val_useis.append(compute_usei_uqi(vrecovered, vsecret))

            # LPIPS (optional)
            if has_lpips and lpips_model is not None:
                try:
                    val_lpips.append(compute_lpips(vstego, vcov, lpips_model))
                except Exception:
                    val_lpips.append(float('nan'))

            # Recovery accuracy (bitstream)
            try:
                acc = batch_bitstream_accuracy(vsecret, vrecovered)
                val_recovery_accs.append(acc)
            except Exception:
                val_recovery_accs.append(float('nan'))

            # Residual bpp estimate
            try:
                bpp = residual_bpp_estimate(vcov, vstego)
                val_bpps.append(bpp)
            except Exception:
                val_bpps.append(float('nan'))

    avg_val_psnr = float(np.nanmean(val_psnrs)) if len(val_psnrs)>0 else 0.0
    avg_val_ssim = float(np.nanmean(val_ssims)) if len(val_ssims)>0 else 0.0
    avg_val_nrpc = float(np.nanmean(val_nrpcs)) if len(val_nrpcs)>0 else float('nan')
    avg_val_usei = float(np.nanmean(val_useis)) if len(val_useis)>0 else float('nan')
    avg_val_lpips = float(np.nanmean(val_lpips)) if len(val_lpips)>0 else float('nan')
    avg_val_recovery_acc = float(np.nanmean(val_recovery_accs)) if len(val_recovery_accs)>0 else float('nan')
    avg_val_bpp = float(np.nanmean(val_bpps)) if len(val_bpps)>0 else float('nan')

    print(f"Epoch {epoch}: Avg Val PSNR = {avg_val_psnr:.3f} dB | SSIM = {avg_val_ssim:.4f} | NRPC = {avg_val_nrpc:.4f} | USEI(UQI) = {avg_val_usei:.4f} | LPIPS = {avg_val_lpips:.4f} | RecAcc = {avg_val_recovery_acc:.4f} | bpp = {avg_val_bpp:.4f}")

    val_psnr_hist.append(avg_val_psnr)
    val_ssim_hist.append(avg_val_ssim)

    per_epoch_metrics.append({
        'epoch': epoch,
        'psnr': avg_val_psnr,
        'ssim': avg_val_ssim,
        'nrpc': avg_val_nrpc,
        'usei': avg_val_usei,
        'lpips': avg_val_lpips,
        'recovery_acc': avg_val_recovery_acc,
        'bpp': avg_val_bpp
    })

    sched_G.step(avg_val_psnr)
    sched_Dec.step(avg_val_psnr)

    # save best
    if avg_val_psnr > best_val_psnr:
        best_val_psnr = avg_val_psnr
        torch.save(G.state_dict(), os.path.join(log_dir, "G_best.pth"))
        torch.save(Dec.state_dict(), os.path.join(log_dir, "Dec_best.pth"))
        torch.save(D.state_dict(), os.path.join(log_dir, "D_best.pth"))
        print(f"Saved best models (PSNR {best_val_psnr:.2f} dB)")

    # periodic checkpoint
    if epoch % save_every == 0:
        torch.save({
            "G": G.state_dict(),
            "Dec": Dec.state_dict(),
            "D": D.state_dict(),
            "opt_G": opt_G.state_dict(),
            "opt_Dec": opt_Dec.state_dict(),
            "opt_D": opt_D.state_dict(),
            "epoch": epoch
        }, os.path.join(log_dir, f"checkpoint_epoch_{epoch}.pth"))

print("Training finished. Best PSNR:", best_val_psnr)

# -----------------------------
# Final Inference / Visualization
# -----------------------------
G.load_state_dict(torch.load(os.path.join(log_dir,"G_best.pth"), map_location=device))
Dec.load_state_dict(torch.load(os.path.join(log_dir,"Dec_best.pth"), map_location=device))
G.eval(); Dec.eval()

sample, _ = next(iter(val_loader))
cover_samp = sample[:8].to(device)
secret_samp = torch.flip(cover_samp, dims=[-1]).to(device)
with torch.no_grad():
    gen_out = G(cover_samp, secret_samp)
    stego_samp = torch.clamp(cover_samp + 0.1 * gen_out, -1, 1)
    recovered_samp = Dec(stego_samp)

# Save sample grid
try:
    grid = make_grid(torch.cat([cover_samp, secret_samp, stego_samp, recovered_samp], dim=0), nrow=8)
    save_path = os.path.join(log_dir, "stegano_results_best.png")
    save_image(((grid+1)/2).clamp(0,1), save_path)
    print("Saved visualization to", save_path)
except Exception as e:
    print("Failed to save sample grid:", e)

# per-epoch metrics CSV
metrics_df = pd.DataFrame(per_epoch_metrics)
metrics_df.to_csv(os.path.join(log_dir, 'per_epoch_metrics.csv'), index=False)

# -----------------------------
# Plotting: one graph per metric (saved individually)
# -----------------------------
# Helper for plotting
def save_metric_plot(x, y, title, ylabel, save_path):
    plt.figure(figsize=(8,5))
    plt.plot(x, y, marker='o')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

epochs_arr = metrics_df['epoch'].tolist()

# PSNR
save_metric_plot(epochs_arr, metrics_df['psnr'].tolist(), 'Validation PSNR across epochs', 'PSNR (dB)', os.path.join(log_dir, 'val_psnr.png'))

# SSIM
save_metric_plot(epochs_arr, metrics_df['ssim'].tolist(), 'Validation SSIM across epochs', 'SSIM', os.path.join(log_dir, 'val_ssim.png'))

# NRPC
save_metric_plot(epochs_arr, metrics_df['nrpc'].tolist(), 'Validation NRPC across epochs', 'NRPC', os.path.join(log_dir, 'val_nrpc.png'))

# USEI / UQI
save_metric_plot(epochs_arr, metrics_df['usei'].tolist(), 'Validation USEI (UQI) across epochs', 'USEI (UQI)', os.path.join(log_dir, 'val_usei.png'))

# LPIPS
save_metric_plot(epochs_arr, metrics_df['lpips'].tolist(), 'Validation LPIPS across epochs', 'LPIPS', os.path.join(log_dir, 'val_lpips.png'))

# Recovery accuracy (bitstream)
save_metric_plot(epochs_arr, metrics_df['recovery_acc'].tolist(), 'Recovery Bitstream Accuracy across epochs', 'Accuracy (fraction)', os.path.join(log_dir, 'val_recovery_acc.png'))

# Residual capacity (bpp)
save_metric_plot(epochs_arr, metrics_df['bpp'].tolist(), 'Estimated Residual Capacity (bpp) across epochs', 'bits per pixel (bpp)', os.path.join(log_dir, 'val_bpp.png'))

# Combined training losses plot (per iteration)
plt.figure(figsize=(14,6))
plt.subplot(1,1,1)
plt.plot(train_G_losses, label='Generator Loss')
plt.plot(train_D_losses, label='Discriminator Loss')
plt.plot(train_Dec_losses, label='Decoder Loss')
plt.title('Training Losses (per iteration)')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(log_dir, 'training_losses_per_iter.png'))
plt.close()

# Histogram of residuals (sample)
try:
    resid = ((stego_samp - cover_samp).detach().cpu().numpy()).flatten()
    plt.hist(resid, bins=100)
    plt.title('Histogram of Residuals (stego - cover) [sample]')
    plt.savefig(os.path.join(log_dir, 'residual_hist.png'))
    plt.close()
except Exception as e:
    print('Failed residual histogram:', e)

print('All plots, CSVs and sample images saved to', log_dir)


Device: cuda img_size: 128
Loaded images: 50000
Models initialized
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/tmp/ipykernel_5156/956504163.py:341: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_G = GradScaler()
/tmp/ipykernel_5156/956504163.py:342: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_D = GradScaler()
/tmp/ipykernel_5156/956504163.py:343: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_Dec = GradScaler()


Loading model from: /home/user7/.local/lib/python3.12/site-packages/lpips/weights/v0.1/vgg.pth
Starting training...


/tmp/ipykernel_5156/956504163.py:381: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_5156/956504163.py:393: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_5156/956504163.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/50: 100%|█████████████████████████████████████████████████████████| 1667/1667 [04:49<00:00,  5.76it/s, D_loss=0.0800, G_loss=20.2044, Dec_loss=3.2343]


Epoch 1: Avg Val PSNR = 20.014 dB | SSIM = 0.6705 | NRPC = 0.9967 | USEI(UQI) = 0.8954 | LPIPS = 0.0473 | RecAcc = 0.5974 | bpp = 10.6102
Saved best models (PSNR 20.01 dB)


...

Epoch 49: Avg Val PSNR = 34.874 dB | SSIM = 0.9629 | NRPC = 0.9999 | USEI(UQI) = 0.9970 | LPIPS = 0.0007 | RecAcc = 0.7255 | bpp = 5.5379


Epoch 50/50: 100%|█████████████████████████████████████████████████████████| 1667/1667 [04:50<00:00,  5.75it/s, D_loss=0.0062, G_loss=1.7635, Dec_loss=0.1813]


Epoch 50: Avg Val PSNR = 34.877 dB | SSIM = 0.9577 | NRPC = 0.9999 | USEI(UQI) = 0.9969 | LPIPS = 0.0007 | RecAcc = 0.7289 | bpp = 5.6439
Training finished. Best PSNR: 35.82306431723115


/tmp/ipykernel_5156/956504163.py:534: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  G.load_state_dict(torch.load(os.path.join(log_dir,"G_best.pth"), map_location=device))
/t

Saved visualization to runs_steg_20251114_114126/stegano_results_best.png
All plots, CSVs and sample images saved to runs_steg_20251114_114126
